You will need to create an account at IRSA/IPAC (https://irsa.ipac.caltech.edu/account/signon/login.do?josso_back_to=https://irsa.ipac.caltech.edu/docs/program_interface/ztf_lightcurve_api.html)



Important links:

https://github.com/MickaelRigault/ztfquery (ztfquery documentation)

https://irsa.ipac.caltech.edu/docs/program_interface/ztf_lightcurve_api.html (ZTF Lightcurve Queries parameters)

https://astroquery.readthedocs.io/en/latest/ipac/irsa/irsa.html (IRSA query)

https://irsa.ipac.caltech.edu/holdings/catalogs.html (IRSA catalog holdings)


In [1]:
#%pip install ztfquery
#%pip install ipympl
#%pip install tqdm
#%pip install openpyxl
#%pip install numpy==1.23

from ztfquery import lightcurve
from tqdm import tqdm

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
file_name= 'teste_sample.xlsx'
radius = 0.0003  # Degrees, limited to (0,0.1667].

# Creating a DataFrame
coord_df = pd.read_excel(file_name)

# Progress bar
pbar = tqdm(total=len(coord_df), desc='Progress', unit='line', leave=False)

# Create a folder named data
if not os.path.exists('data'):
    os.makedirs('data')

plots_path = os.path.join('data', 'plots')
if not os.path.exists(plots_path):
    os.makedirs(plots_path)

while True:
    try:
        for index, row in coord_df.iterrows():
                
            ra = row['ra']  #Degrees
            dec = row['dec']  #Degreees

            #----------------------------------------------------------------------------# g-band
            datag = lightcurve.LCQuery.download_data(circle=[ra,dec,radius], bandname="g")
            tableg = datag[['mjd','mag', 'magerr']] 

            if len(tableg)>0:
                mean_magerrg = np.mean(tableg['magerr'])
                mean_magg = np.mean(tableg['mag'])
                rmsg = np.sqrt(np.mean((tableg['mag'] - mean_magg) ** 2))
                inf_g = mean_magg - 0.7
                sup_g = mean_magg + 0.7
            else:
                mean_magerrg = 1.0
                mean_magg = 1.0
                rmsg = 1.0
                inf_g = 1.0
                sup_g = 1.0


            #----------------------------------------------------------------------------# r-band
            datar = lightcurve.LCQuery.download_data(circle=[ra,dec,radius], bandname="r")
            tabler = datar[['mjd','mag', 'magerr']] 

            if len(tabler)>0:
                mean_magerrr = np.mean(tabler['magerr'])
                mean_magr = np.mean(tabler['mag'])
                rmsr = np.sqrt(np.mean((tabler['mag'] - mean_magr) ** 2))
                inf_r = mean_magr - 0.7
                sup_r = mean_magr + 0.7            
            else:
                mean_magerrr = 1.0
                mean_magr = 1.0
                rmsr = 1.0
                inf_r = 1.0
                sup_r = 1.0   


            #----------------------------------------------------------------------------# i-band
            datai = lightcurve.LCQuery.download_data(circle=[ra,dec,radius], bandname="i")
            tablei = datai[['mjd','mag', 'magerr']] 

            if len(tablei)>0:
                mean_magerri = np.mean(tablei['magerr'])
                mean_magi = np.mean(tablei['mag'])
                rmsi = np.sqrt(np.mean((tablei['mag'] - mean_magi) ** 2))
                inf_i = mean_magi - 0.7
                sup_i = mean_magi + 0.7            
            else:
                mean_magerri = 1.0
                mean_magi = 1.0
                rmsi = 1.0
                inf_i = 1.0
                sup_i = 1.0 

            pbar.update(1)
  
            # Creating a folder for each coordinate
            pasta_ra_dec = f'{ra}_{dec}'
            pasta_path = os.path.join('data', pasta_ra_dec)

            if not os.path.exists(pasta_path):
                os.makedirs(pasta_path)
            
            tableg.to_csv(os.path.join(pasta_path, f'{pasta_ra_dec}_g.dat'), sep='\t', index=False)
            tabler.to_csv(os.path.join(pasta_path, f'{pasta_ra_dec}_r.dat'), sep='\t', index=False)
            tablei.to_csv(os.path.join(pasta_path, f'{pasta_ra_dec}_i.dat'), sep='\t', index=False)

            # Figure Plot
            fig, axs = plt.subplots(3, 1, figsize=(10, 13))
    
            axs[0].errorbar(tableg['mjd'], tableg['mag'], yerr=tableg['magerr'], label='g-band', fmt='o',ecolor='darkgrey',markeredgecolor='darkgrey',markersize =7, color='r') 
            axs[0].legend(loc='upper right', fontsize=12, title=f'rms: {rmsg:.4f}\nreliability: {rmsg/mean_magerrg:.2f}')
            axs[0].set_xlabel('MJD')
            axs[0].set_ylabel('Magnitude')
            axs[0].set_ylim(inf_g, sup_g)
            axs[0].invert_yaxis()
            axs[0].set_title((ra,dec)) 
    
            axs[1].errorbar(tabler['mjd'], tabler['mag'], yerr=tabler['magerr'], label='r-band', fmt='o',ecolor='darkgrey',markeredgecolor='darkgrey',markersize =7, color='g')
            axs[1].legend(loc='upper right', fontsize=12, title=f'rms: {rmsr:.4f}\nreliability: {rmsr/mean_magerrr:.2f}')
            axs[1].set_xlabel('MJD')
            axs[1].set_ylabel('Magnitude')
            axs[1].set_ylim(inf_r, sup_r)
            axs[1].invert_yaxis()
            axs[1].set_title((ra,dec)) 
    
            axs[2].errorbar(tablei['mjd'], tablei['mag'], yerr=tablei['magerr'], label='i-band', fmt='o',ecolor='darkgrey',markeredgecolor='darkgrey',markersize =7, color='darkorange')
            axs[2].legend(loc='upper right', fontsize=12, title=f'rms: {rmsi:.4f}\nreliability: {rmsi/mean_magerri:.2f}')
            axs[2].set_xlabel('MJD')
            axs[2].set_ylabel('Magnitude')
            axs[2].set_ylim(inf_i, sup_i)
            axs[2].invert_yaxis()
            axs[2].set_title((ra,dec)) 
 
            plt.tight_layout()
            plt.savefig(os.path.join(plots_path, f'{rmsg:.2f}-ra{ra},dec{dec}.png'), facecolor='white') #save all plots in the same folder
            plt.close()  
        break
    except KeyboardInterrupt: #Crtl + c to break
        break
pbar.close()

Progress:   0%|          | 0/5 [00:00<?, ?line/s]